<a href="https://colab.research.google.com/github/codewithlax/NLP-and-LLMs/blob/main/Fake_News_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("fake_news_train.csv",error_bad_lines=False, engine="python")
df.head()

<ipython-input-2-6ac97b479e04>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv("fake_news_train.csv",error_bad_lines=False, engine="python")
Skipping line 2826: unexpected end of data


,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
df.isnull().sum()

id          0
title      76
author    284
text        7
label       0
dtype: int64

In [ ]:
df = df.dropna()
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [ ]:
df.shape

(2464, 5)

In [ ]:
X = df.drop("label",axis=1)
X.shape

(2464, 4)

In [ ]:
y = df["label"]
y.shape

(2464,)

In [ ]:
import tensorflow as tf
tf.__version__

'2.12.0'

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [ ]:
voc_size = 5000
messages = X.copy()

In [ ]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
messages.reset_index(inplace=True)
# Data Preprocessing:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
  review = re.sub('[^a-zA-Z]',' ', messages['title'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word)for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [ ]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [ ]:
one_hot_encodeed_corpus = [one_hot(words,voc_size) for words in corpus]
one_hot_encodeed_corpus

[[3869, 4117, 2418, 4052, 1480, 3784, 1010, 4371, 1303, 3457],
 [196, 935, 1804, 2110, 2315, 807, 669],
 [3536, 1438, 2634, 4739],
 [2001, 1726, 4430, 4109, 31, 116],
 [4506, 2315, 2942, 2310, 1414, 1700, 2315, 361, 4480, 4746],
 [3794,
  3973,
  2055,
  665,
  539,
  4529,
  3469,
  2100,
  3422,
  2169,
  2087,
  4815,
  3392,
  2909,
  669],
 [531, 2330, 1769, 3200, 1566, 4591, 3081, 2561, 1992, 4413, 1856],
 [3646, 2986, 4611, 1037, 2310, 4278, 4529, 2793, 1992, 4413, 1856],
 [497, 159, 4805, 2980, 3184, 4621, 4378, 38, 4529, 3340],
 [2716, 3372, 853, 4308, 1392, 2694, 2127, 701],
 [2108, 3824, 938, 1479, 642, 2822, 1886, 4434, 4065, 4060, 105],
 [4109, 2905, 1480, 4621, 4529, 2310],
 [2525, 2862, 2520, 1302, 243, 2002, 1361, 1811, 3532],
 [2870, 2231, 2338, 2460, 4245, 2212, 4866, 1992, 4413, 1856],
 [4071, 1184, 800, 484, 2046, 1992, 4413, 1856],
 [249, 2911, 1129, 2588, 3414, 2365, 3342, 873, 1585, 1551],
 [2584, 3185, 935],
 [1907, 4743, 4942, 4873, 4529, 3978, 2300, 669],
 [31

In [ ]:
max_length = 20
padded_corpus = pad_sequences(one_hot_encodeed_corpus,padding="pre",maxlen=max_length)
padded_corpus

array([[   0,    0,    0, ..., 4371, 1303, 3457],
       [   0,    0,    0, ..., 2315,  807,  669],
       [   0,    0,    0, ..., 1438, 2634, 4739],
       ...,
       [   0,    0,    0, ..., 3176, 1996,  669],
       [   0,    0,    0, ..., 1992, 4413, 1856],
       [   0,    0,    0, ...,  557,  610, 4760]], dtype=int32)

In [ ]:
embedding_vector_dimensions = 10 # Dimensions
# Embedding Layer and LSTM:
model = Sequential()
model.add(Embedding(voc_size,embedding_vector_dimensions,input_length=max_length))
model.add(LSTM(50))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss='binary_crossentropy', optimizer ='adam', metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 10)            50000     
                                                                 
 lstm (LSTM)                 (None, 50)                12200     
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 62,251
Trainable params: 62,251
Non-trainable params: 0
_________________________________________________________________


In [ ]:
import numpy as np
X_final=np.array(padded_corpus)
y_final=np.array(y)
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_final, y_final, test_size=0.3, random_state=42)

In [ ]:
history = model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs = 10, batch_size=64)

Epoch 1/10
27/27 [==============================] - 14s 240ms/step - loss: 0.6802 - accuracy: 0.5557 - val_loss: 0.6540 - val_accuracy: 0.5743
Epoch 2/10
27/27 [==============================] - 4s 149ms/step - loss: 0.5927 - accuracy: 0.6862 - val_loss: 0.5551 - val_accuracy: 0.7905
Epoch 3/10
27/27 [==============================] - 1s 55ms/step - loss: 0.4477 - accuracy: 0.8428 - val_loss: 0.4329 - val_accuracy: 0.8135
Epoch 4/10
27/27 [==============================] - 1s 47ms/step - loss: 0.2938 - accuracy: 0.8852 - val_loss: 0.3687 - val_accuracy: 0.8324
Epoch 5/10
27/27 [==============================] - 2s 64ms/step - loss: 0.1681 - accuracy: 0.9490 - val_loss: 0.3973 - val_accuracy: 0.8324
Epoch 6/10
27/27 [==============================] - 2s 84ms/step - loss: 0.1087 - accuracy: 0.9658 - val_loss: 0.3897 - val_accuracy: 0.8446
Epoch 7/10
27/27 [==============================] - 1s 21ms/step - loss: 0.0583 - accuracy: 0.9849 - val_loss: 0.4273 - val_accuracy: 0.8527
Epoch 8/10

In [ ]:
y_pred = model.predict(X_test)

24/24 [==============================] - 1s 3ms/step


In [ ]:
y_pred = np.where(y_pred>0.5,1,0) # Works like conditional operator

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
confusion_matrix(y_test,y_pred)

array([[386,  39],
       [ 80, 235]])

In [ ]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.83      0.91      0.87       425
           1       0.86      0.75      0.80       315

    accuracy                           0.84       740
   macro avg       0.84      0.83      0.83       740
weighted avg       0.84      0.84      0.84       740

